In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [45]:
import tensorflow as tf

import keras

from keras.utils.np_utils import to_categorical

import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential

from keras.layers import Dense,Conv2D,MaxPooling2D

from keras.layers import Flatten

from random import seed

from random import randint

import random

from keras.preprocessing.image import ImageDataGenerator

from keras.layers import Dropout,ZeroPadding2D,AveragePooling2D



# Data import

In [46]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")
Y_train = train["label"]

# Drop 'label' column
X_train = train.drop(labels = ["label"],axis = 1) 

# free some space
del train 

In [47]:
X_train = X_train / 255.0
test = test / 255.0

In [48]:
X_train = X_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

In [49]:
Y_train = to_categorical(Y_train, num_classes = 10)

In [50]:
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1)

In [ ]:
print(Y_train.shape)

# Logit Perturbation

In [51]:
mean=0

sd=1

In [52]:
ep = np.random.normal(mean, sd, 1000)

In [54]:
l=[]

c=0

seed(1)

while c<591:

    i=0
    
    l.append(random.sample(range(i,i+65),10))
    
    i+=64

    c+=1

# Student Network

In [55]:
smodel=Sequential()

smodel.add(Dense(800,activation='linear',input_dim=500))

smodel.add(Dense(800,activation='linear'))

smodel.add(Dense(10,activation='linear'))

In [56]:
smodel.compile(optimizer="sgd",loss="mse")

In [66]:
smodel.fit(a,Y_train,batch_size=64)

591/591 [==============================] - 1s 2ms/step - loss: 0.0951


# Teacher Network

In [58]:
model = Sequential()

model.add(Conv2D(20,kernel_size=[5,5],activation='tanh', input_shape = (28,28,1)))

model.add(MaxPooling2D(strides=(2,2)))

model.add(Conv2D(50,kernel_size=[5,5],activation='tanh'))

model.add(MaxPooling2D(strides=(2,2)))

model.add(Flatten())

model.add(Dense(500,activation='tanh'))

model.add(Dense(10,activation='softmax'))


In [59]:
layer_output=[layer.output for layer in model.layers[:5]]

In [60]:
model.compile(loss= "mse",optimizer="sgd")



In [61]:
model.fit(X_train,Y_train,batch_size=64)

591/591 [==============================] - 2s 2ms/step - loss: 0.0896


In [ ]:
model.summary()

In [62]:
model.layers[5]._name='conv'

In [63]:
layer_output=model.get_layer('conv').output

inter_model=keras.models.Model(inputs=model.input,outputs=layer_output)



In [64]:
a=np.zeros(shape=(37800,1,500))

In [65]:
for i in range(37800):
    
    

    a[i]=inter_model.predict(X_train[i].reshape(1,28,28,1))
    
    if i in l:
    
        a[i]=a[i]+ep*a[i]
    
    i=i+1

## Test

In [67]:
len(X_val)

4200

In [68]:
a=np.zeros(shape=(len(X_val),1,500))

In [70]:
for i in range(4200):
    
    a[i]=inter_model.predict(test[i].reshape(1,28,28,1))
    
    i=i+1
    
    

In [72]:
smodel.evaluate(a,Y_val)

NameError: name 'y_val' is not defined